In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [10]:
dataset_path = '/content/HAPPYSAD'

image_size = (128, 128)
batch_size = 32

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    directory=f'{dataset_path}/train',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 555 images belonging to 2 classes.


In [18]:
val_generator = val_datagen.flow_from_directory(
    directory = f'{dataset_path}/valid',
    target_size = image_size,
    batch_size = batch_size,
    class_mode ='binary'
)

Found 53 images belonging to 2 classes.


In [20]:
test_generator = test_datagen.flow_from_directory(
    directory = f'{dataset_path}/valid',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = False
)

Found 53 images belonging to 2 classes.


In [22]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [23]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs=20,
    callbacks=[early_stop]
)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9842 - loss: 0.0434 - val_accuracy: 0.9434 - val_loss: 0.2211
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9868 - loss: 0.0281 - val_accuracy: 0.9057 - val_loss: 0.4252
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9891 - loss: 0.0280 - val_accuracy: 0.9434 - val_loss: 0.2504


In [29]:
model.evaluate(test_generator)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9623 - loss: 0.1629


[0.22114624083042145, 0.9433962106704712]

In [34]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/deneme foto.jpg"
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("Sad😢")
else:
    print("Happy🙂")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Happy🙂


In [35]:
model.save("happysad_model.h5")
